# Lab2: Lemmatization and full text search (FTS)

In [1]:
import elasticsearch
import elasticsearch_dsl
import os
import re

In [2]:
ACT_DIRECTORY = '../files'

## 1. Install ElasticSearch (ES).

Zamiast lokalnej instancji ES wykorzystany został obraz Dockerowy.

In [3]:
es = elasticsearch.Elasticsearch()
elasticsearch_dsl.connections.add_connection('python_client', es)
es.info()

{'name': '15f0fe802b8f',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': '-gSQyVHTT1adZQVHiafp8Q',
 'version': {'number': '7.9.2',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'd34da0ea4a966c4e49417f2da2f244e3e97b4e6e',
  'build_date': '2020-09-23T00:45:33.626720Z',
  'build_snapshot': False,
  'lucene_version': '8.6.2',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

## 2. Install an ES plugin for Polish https://github.com/allegro/elasticsearch-analysis-morfologik

Plugin Morfologik został zainstalowany zgodnie z instrukcją.

In [4]:
print(es.cat.plugins(params={'v': '', 'h': 'name,component,version,description'}))

name         component           version description
15f0fe802b8f analysis-morfologik 7.9.2   Morfologik Polish Lemmatizer plugin for Elasticsearch



## 3. Define an ES analyzer for Polish texts containing:
* standard tokenizer
* synonym filter with the following definitions:
  * kpk - kodeks postępowania karnego
  * kpc - kodeks postępowania cywilnego
  * kk - kodeks karny
  * kc - kodeks cywilny
* Morfologik-based lemmatizer
* lowercase filter

Zdefiniowany został analizator zgodnie z wyznacznymi z polecenia.

In [5]:
analyzer = elasticsearch_dsl.analyzer(
    'act_analyzer',
    type='custom',
    tokenizer='standard',                             # standard tokenizer
    filter=[
        'lowercase',                                  # lowercase filter
        elasticsearch_dsl.analysis.token_filter(      # synonym filter
            'synonym_filter', 
            type='synonym', 
            synonyms=[
                'kpk,kodeks postępowania karnego',
                'kpc,kodeks postępowania cywilnego',
                'kk,kodeks karny',
                'kc,kodeks cywilny',
            ],
        ), 
        'morfologik_stem',                            # Morfologik-based lemmatizer
    ],
)

## 4. Define an ES index for storing the contents of the legislative acts.

Przygotowana została klasa wrappująca analizowane dokumenty. Oprócz treści pliku uwzględniona została również nazwa pliku oraz rok, z którego pochodzi dana ustawa.

In [6]:
class Act(elasticsearch_dsl.Document):
    file_name = elasticsearch_dsl.Text()
    year = elasticsearch_dsl.Integer()
    content = elasticsearch_dsl.Text(
        analyzer=analyzer,
    )
    
    class Index:
        name = 'acts'

Indeks został zainizjalizowany.

In [7]:
if Act._index.exists(using=es):
    Act._index.delete(using=es)

Act.init(using=es)

## 5. Load the data to the ES index.

Do wczytania zawartości plików wykorzystany został kod z poprzednich zajęć. Każdy plik został wczytany do indeksu.

In [8]:
HEADER = r'u\s*s\s*t\s*a\s*w\s*a[\s\S]*?z\s*dnia\s*\d{1,2}\s*\w*\s*\d{4}'

for root, _, files in os.walk(ACT_DIRECTORY):
    for file_name in files:
        path = os.path.join(root, file_name)
        with open(path, encoding='utf-8') as file:
            content = file.read()
            date = re.findall(HEADER, content, re.IGNORECASE)
            year = date[0][-4:] if date else os.path.splitext(file_name)[0].split('_')[0]
            act = Act(file_name=file_name, year=year, content=content)
            act.save(using=es)

W celu weryfikacji liczba dokumentów w indeksie została porównana z liczbą plików w korpusie.

In [9]:
es.search(index='acts')['hits']['total']['value'] == len(os.listdir(ACT_DIRECTORY))

True

## 6. Determine the number of legislative acts containing the word ustawa (in any form).

Liczba plików zawierających słowo "ustawa" to 1179, co oznacza, że tylko jeden z plików nie zawiera tego słowa.

In [10]:
search = elasticsearch_dsl.Search(index='acts') \
    .using(es) \
    .query('match', content='ustawa')

results = search[:1500].execute()

print(f'The number of files containing the word "ustawa" is: {len(results)}')

The number of files containing the word "ustawa" is: 1179


## 7. Determine the number of legislative acts containing the words kodeks postępowania cywilnego in the specified order, but in an any inflection form.

Liczba plików zawierających słowa "kodeks postępowania cywilnego" to 100. Dla porównania zliczone zostały również wystąpienia "kpc" (czyli stringa, który powinien zostać potraktowany jako synonim) -- wynik również wyniósł 100.

In [11]:
search = elasticsearch_dsl.Search(index='acts') \
    .using(es) \
    .query('match_phrase', content='kodeks postępowania cywilnego')

results = search[:1500].execute()

print(f'The number of files containing the words "kodeks postępowania cywilnego" is: {len(results)}')

The number of files containing the words "kodeks postępowania cywilnego" is: 100


In [16]:
search = elasticsearch_dsl.Search(index='acts') \
    .using(es) \
    .query('match_phrase', content='kpc')

results = search[:1500].execute()

print(f'The number of files containing the words "kpc" is: {len(results)}')

The number of files containing the words "kpc" is: 100


## 8. Determine the number of legislative acts containing the words wchodzi w życie (in any form) allowing for up to 2 additional words in the searched phrase.

Liczba plików zawierających słowa "wchodzi w życie" to 1175, co oznacza, że pięć plików nie zawiera tych słów.

In [12]:
search = elasticsearch_dsl.Search(index='acts') \
    .using(es) \
    .query(elasticsearch_dsl.query.MatchPhrase(content={'query': 'wchodzi w życie', 'slop': 2}))

results = search[:1500].execute()

print(f'The number of files containing the words "wchodzi w życie" is: {len(results)}')

The number of files containing the words "wchodzi w życie" is: 1175


## 9. Determine the 10 documents that are the most relevant for the phrase konstytucja.

In [13]:
search = elasticsearch_dsl.Search(index='acts') \
    .using(es) \
    .query(elasticsearch_dsl.query.Match(content={'query': 'konstytucja'}))

results = search[:10].execute()

file_names = [hit.file_name for hit in results.hits]

print(f'The top 10 files containing the phrase "konstytucja" are: {", ".join(file_names)}.')

The top 10 files containing the phrase "konstytucja" are: 1997_629.txt, 2000_443.txt, 1997_604.txt, 1996_350.txt, 1997_642.txt, 2001_23.txt, 1996_199.txt, 1999_688.txt, 1997_681.txt, 2001_1082.txt.


## 10. Print the excerpts containing the word konstytucja (up to three excerpts per document) from the previous task.

W wyprintowanych fragmentach widać, że zliczone zostały różne formy słowa "konstytucja", co sugeruje, że działanie analizatora jest zgodne z oczekiwanym.

In [14]:
search = elasticsearch_dsl.Search(index='acts') \
    .using(es) \
    .query(elasticsearch_dsl.query.Match(content={'query': 'konstytucja'})) \
    .highlight('content', number_of_fragments=3)

results = search[:10].execute()

print(f'The top 10 files containing the phrase "konstytucja" with selected excerpts:')
for index, hit in enumerate(results.hits):
    print(f'{index +  1}. {file_name}:')
    for highlight in hit.meta.highlight.content:
        print(f'  - {highlight}')


The top 10 files containing the phrase "konstytucja" with selected excerpts:
1. 1997_496.txt:
  - o zmianie ustawy konstytucyjnej o trybie przygotowania
           i uchwalenia <em>Konstytucji</em> Rzeczypospolitej
  - W ustawie  konstytucyjnej z  dnia 23 kwietnia 1992 r. o trybie przygotowania i 
uchwalenia <em>Konstytucji</em>
  - Do zgłoszenia projektu <em>Konstytucji</em> załącza się wykaz 
                obywateli popierających zgłoszenie
2. 1997_496.txt:
  - umowy międzynarodowej i nie wypełnia przesłanek określonych w art. 89
     ust. 1 lub art. 90 <em>Konstytucji</em>
  - międzynarodowej lub załącznika nie
     wypełnia przesłanek określonych w art. 89 ust. 1 lub art. 90 <em>Konstytucji</em>
  - co do zasadności wyboru
  trybu ratyfikacji umowy międzynarodowej, o którym mowa w art. 89 ust. 2
  <em>Konstytucji</em>
3. 1997_496.txt:
  - Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności celów partii 
   politycznej z <em>Konstytucją</em>
  - Jeżeli Trybunał Konstytucy